In [5]:
from data_preprocessing import get_dataloaders
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# ==========================================
# 1. CONFIGURATION
# ==========================================
MODEL_SAVE_PATH = "mobilenet_v2_stanford_cars.pth"
BATCH_SIZE = 32
NUM_CLASSES = 196
NUM_EPOCHS = 30
IMG_SIZE = 224  # Image resolution for MobileNetV2

# ==========================================
# 2. MODEL ARCHITECTURE
# ==========================================
def get_mobilenet_model(num_classes=196):
    # Load Pre-trained MobileNetV2
    model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)

    # --- STRATEGY: FINE-TUNING ---
    # 1. Freeze the early layers (generic features like lines/edges)
    for param in model.parameters():
        param.requires_grad = False
        
    # 2. Unfreeze the last few inverted residual blocks for better feature learning
    # MobileNetV2 has features organized in a Sequential container
    # Unfreeze the last 4 blocks (out of 19 total blocks)
    for param in model.features[-4:].parameters():
        param.requires_grad = True
        
    # 3. Replace the Classifier Head with Higher Dropout
    in_features = model.classifier[1].in_features  # MobileNetV2 has 1280 features
    model.classifier = nn.Sequential(
        nn.Dropout(0.3),
        nn.Linear(in_features, 1024),  # Add an intermediate layer
        nn.ReLU(),
        nn.Dropout(0.5),                # INCREASED dropout
        nn.Linear(1024, num_classes)
    )

    return model

# ==========================================
# 3. TRAINING SKELETON
# ==========================================

if __name__ == "__main__":
    
    print("Initializing Hugging Face Data Pipeline...")
    
    try:
        # 1. Load Data using HF pipeline (handles downloading, splitting, and transforms)
        print("Loading data from Hugging Face Hub...")
        train_dl, val_dl, test_dl = get_dataloaders(
            batch_size=BATCH_SIZE, 
            img_size=IMG_SIZE, 
            num_workers=0
        )
        
        print("Initializing MobileNetV2...")
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = get_mobilenet_model(num_classes=NUM_CLASSES).to(device)
        
        # Hyperparameters
        criterion = nn.CrossEntropyLoss()
        # Add weight_decay=1e-4
        optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
        
        # Learning Rate Scheduler
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)
        
        print(f"Starting Training on {device}...")
        
        for epoch in range(NUM_EPOCHS):
            model.train()
            running_loss = 0.0
            correct_predictions = 0 
            total_samples = 0
            
            for images, labels in train_dl:
                images, labels = images.to(device), labels.to(device)
                
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
                running_loss += loss.item()
                
                # Track Accuracy during training to see progress
                _, predicted = torch.max(outputs, 1)
                total_samples += labels.size(0)
                correct_predictions += (predicted == labels).sum().item()

            epoch_loss = running_loss / len(train_dl)
            epoch_acc = 100 * correct_predictions / total_samples
            
            print(f"Epoch {epoch+1}/{NUM_EPOCHS} | Loss: {epoch_loss:.4f} | Accuracy: {epoch_acc:.2f}%")
            
            # Update LR based on training loss (or validation loss if we computed it per epoch)
            scheduler.step(epoch_loss)
            
        print("Training loop finished successfully!")
        print("Saving model...")
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        print(f"Model saved as '{MODEL_SAVE_PATH}'")

    except Exception as e:
        print("\n An error occurred during execution:")
        print(e)
        import traceback
        traceback.print_exc()

Initializing Hugging Face Data Pipeline...
Loading data from Hugging Face Hub...
 Loading 'tanganke/stanford_cars' from Hugging Face Hub...
✅ Data Split: 6515 Train | 1629 Val | 8041 Test
Initializing MobileNetV2...
Starting Training on cuda...
Epoch 1/30 | Loss: 5.2212 | Accuracy: 1.66%
Epoch 2/30 | Loss: 4.6394 | Accuracy: 5.94%
Epoch 3/30 | Loss: 3.9662 | Accuracy: 12.17%
Epoch 4/30 | Loss: 3.3868 | Accuracy: 19.36%
Epoch 5/30 | Loss: 2.9395 | Accuracy: 26.91%
Epoch 6/30 | Loss: 2.5770 | Accuracy: 33.32%
Epoch 7/30 | Loss: 2.2856 | Accuracy: 39.05%
Epoch 8/30 | Loss: 2.0235 | Accuracy: 45.85%
Epoch 9/30 | Loss: 1.8130 | Accuracy: 49.79%
Epoch 10/30 | Loss: 1.6460 | Accuracy: 54.34%
Epoch 11/30 | Loss: 1.4887 | Accuracy: 58.48%
Epoch 12/30 | Loss: 1.3422 | Accuracy: 62.00%
Epoch 13/30 | Loss: 1.2460 | Accuracy: 64.48%
Epoch 14/30 | Loss: 1.1298 | Accuracy: 67.63%
Epoch 15/30 | Loss: 1.0427 | Accuracy: 69.58%
Epoch 16/30 | Loss: 0.9540 | Accuracy: 72.00%
Epoch 17/30 | Loss: 0.8931 | A